In [1]:
import pandas as pd
import numpy as np
from systems.vault import Vault, RMS
from config import config_dict
import logging
from utils import create_logger
logger = create_logger(log_level=logging.DEBUG, logger_name='datafetcher', print_to_console=True)
import warnings
warnings.filterwarnings("ignore")

In [2]:
# Datafeeder starting parameters (right now it'll be hardcoded, and later, it will be fetched from a datafeeder_config variable)
data_inputs = {'1min':{'columns':['open', 'high', 'low', 'close', 'volume,' 'SMA15', 'SMA30'], 'lookback':100}, '1d':{'columns':['open', 'high', 'low', 'close', 'volume,' 'SMA15', 'SMA30'], 'lookback':100}}
tickers = ['AAPL', 'MSFT', 'NVDA']

datafeeder_config = {'data_inputs':data_inputs, 'tickers':tickers}

In [3]:
# Create date ranges
date_range_1min = pd.date_range(start='2023-01-01', periods=100, freq='min')
date_range_1d = pd.date_range(start='2023-01-01', periods=200, freq='D')

# Generate random prices and other data
np.random.seed(0)

def generate_data(date_range, granularity):
    data = []
    for date in date_range:
        for ticker in tickers:
            open_price = np.random.uniform(100, 200)
            high_price = open_price + np.random.uniform(0, 10)
            low_price = open_price - np.random.uniform(0, 10)
            close_price = np.random.uniform(low_price, high_price)
            volume = np.random.randint(1000, 10000)
            sma15 = np.random.uniform(100, 200)
            sma30 = np.random.uniform(100, 200)
            data.append([granularity, date, ticker, open_price, high_price, low_price, close_price, volume, sma15, sma30])
    return data

data_1min = generate_data(date_range_1min, '1min')
data_1d = generate_data(date_range_1d, '1d')

# Create DataFrames
columns = ['Granularity', 'Datetime', 'Ticker', 'Open', 'High', 'Low', 'Close', 'Volume', 'SMA15', 'SMA30']
df_1min = pd.DataFrame(data_1min, columns=columns)
df_1d = pd.DataFrame(data_1d, columns=columns)

# Concatenate DataFrames
df = pd.concat([df_1min, df_1d])

# Set multi-index
df.set_index(['Granularity', 'Datetime', 'Ticker'], inplace=True)

# Pivot the DataFrame to have tickers as columns
df = df.unstack(level='Ticker')
market_data_df = df
market_data_df

Open                          \
Ticker                                 AAPL        MSFT        NVDA   
Granularity Datetime                                                  
1d          2023-01-01 00:00:00  149.723140  108.154122  181.968981   
            2023-01-02 00:00:00  180.921190  118.473854  183.417626   
            2023-01-03 00:00:00  174.370511  123.718963  108.099818   
            2023-01-04 00:00:00  140.902151  150.676974  117.610384   
            2023-01-05 00:00:00  143.593128  198.186422  125.664751   
...                                     ...         ...         ...   
1min        2023-01-01 01:35:00  115.148601  112.961905  121.151798   
            2023-01-01 01:36:00  137.558856  106.290696  181.151847   
            2023-01-01 01:37:00  184.267801  103.921054  198.114177   
            2023-01-01 01:38:00  126.798868  164.364401  139.618456   
            2023-01-01 01:39:00  162.911562  193.345589  196.828552   

                                       High                          \
Ticker                                 AAPL        MSFT        NVDA   
Granularity Datetime                                                  
1d          2023-01-01 00:00:00  153.643357  116.464486  186.853405   
            2023-01-02 00:00:00  182.630538  123.247377  185.721629   
            2023-01-03 00:00:00  178.869307  125.554008  115.136322   
            2023-01-04 00:00:00  144.224640  158.941754  120.808457   
            2023-01-05 00:00:00  149.975319  202.468408  133.074990   
...                                     ...         ...         ...   
1min        2023-01-01 01:35:00  116.769331  113.009779  129.461280   
            2023-01-01 01:36:00  141.149921  115.834705  185.912687   
            2023-01-01 01:37:00  185.904532  104.173372  199.621328   
            2023-01-01 01:38:00  127.417760  168.583223  145.637893   
            2023-01-01 01:39:00  165.244272  195.874128  200.358648   

                                        Low                          \
Ticker                                 AAPL        MSFT        NVDA   
Granularity Datetime                                                  
1d          2023-01-01 00:00:00  148.283374  106.433700  180.626310   
            2023-01-02 00:00:00  172.264424  114.910427  179.150712   
            2023-01-03 00:00:00  170.248699  116.171124  101.628565   
            2023-01-04 00:00:00  131.006900  140.852510  109.442132   
            2023-01-05 00:00:00  136.852356  189.375756  123.910202   
...                                     ...         ...         ...   
1min        2023-01-01 01:35:00  105.513901  107.424244  120.459950   
            2023-01-01 01:36:00  131.392672  104.801110  175.920287   
            2023-01-01 01:37:00  181.665276  101.205551  196.491659   
            2023-01-01 01:38:00  118.585833  159.593407  138.984766   
            2023-01-01 01:39:00  157.752828  185.767348  195.233586   

                                      Close  ...             Volume        \
Ticker                                 AAPL  ...        NVDA   AAPL  MSFT   
Granularity Datetime                         ...                            
1d          2023-01-01 00:00:00  152.597211  ...  185.923252   6353  2241   
            2023-01-02 00:00:00  178.333212  ...  183.162189   9978  5227   
            2023-01-03 00:00:00  171.481263  ...  106.639582   6400  4187   
            2023-01-04 00:00:00  139.524618  ...  115.574681   4068  2795   
            2023-01-05 00:00:00  141.088098  ...  128.235667   8573  9533   
...                                     ...  ...         ...    ...   ...   
1min        2023-01-01 01:35:00  108.946404  ...  128.261429   5970  7555   
            2023-01-01 01:36:00  140.178198  ...  178.423589   6351  6708   
            2023-01-01 01:37:00  183.238961  ...  196.850572   3042  6245   
            2023-01-01 01:38:00  121.939020  ...  141.711600   8452  9371   
            2023-01-01 01:39:00  159.550982  ...  196

In [7]:

vault = Vault(config_dict)
logger.debug({'datafeeder_config':vault.datafeeder_config})
# Load dummy data and check if the signals are generated.
#market_data_df = pd.DataFrame()
signals_output = vault.generate_signals(market_data_df)
logger.debug({'signals_output':signals_output})
#rms = RMS()
#orders = rms.convert_signals_to_orders(signals_output)
#logger.debug({'orders':orders})

2024-09-12 22:16:19,282 - datafetcher - DEBUG - {'datafeeder_config': {'data_inputs': {'strategy_1': {'granularity': '1min', 'lookback': 100, 'columns': ['Open', 'High', 'Low', 'Close', 'Volume']}, 'strategy_2': {'granularity': '1d', 'lookback': 100, 'columns': ['Open', 'High', 'Low', 'Close', 'Volume']}}, 'tickers': ['AAPL', 'MSFT', 'NVDA']}}
2024-09-12 22:16:19,306 - datafetcher - DEBUG - {'signals_output': {'signals': [{'symbol': 'AAPL', 'signal_strength': 0.8, 'strategy_name': 'strategy_1', 'timestamp': Timestamp('2023-01-01 01:39:00'), 'entry_order_type': 'market', 'exit_order_type': 'stoploss_pct'}, {'symbol': 'MSFT', 'signal_strength': 0.8, 'strategy_name': 'strategy_1', 'timestamp': Timestamp('2023-01-01 01:39:00'), 'entry_order_type': 'market', 'exit_order_type': 'stoploss_pct'}, {'symbol': 'NVDA', 'signal_strength': 0.8, 'strategy_name': 'strategy_1', 'timestamp': Timestamp('2023-01-01 01:39:00'), 'entry_order_type': 'market', 'exit_order_type': 'stoploss_pct'}], 'ideal_portf

In [8]:
signals_output

{'signals': [{'symbol': 'AAPL',
   'signal_strength': 0.8,
   'strategy_name': 'strategy_1',
   'timestamp': Timestamp('2023-01-01 01:39:00'),
   'entry_order_type': 'market',
   'exit_order_type': 'stoploss_pct'},
  {'symbol': 'MSFT',
   'signal_strength': 0.8,
   'strategy_name': 'strategy_1',
   'timestamp': Timestamp('2023-01-01 01:39:00'),
   'entry_order_type': 'market',
   'exit_order_type': 'stoploss_pct'},
  {'symbol': 'NVDA',
   'signal_strength': 0.8,
   'strategy_name': 'strategy_1',
   'timestamp': Timestamp('2023-01-01 01:39:00'),
   'entry_order_type': 'market',
   'exit_order_type': 'stoploss_pct'}],
 'ideal_portfolios': [{'symbols': {'NVDA': -1, 'AAPL': -1, 'MSFT': -1},
   'strategy_name': 'strategy_2',
   'timestamp': Timestamp('2023-07-19 00:00:00')}]}

In [9]:
vault.datafeeder_config

{'data_inputs': {'strategy_1': {'granularity': '1min',
   'lookback': 100,
   'columns': ['Open', 'High', 'Low', 'Close', 'Volume']},
  'strategy_2': {'granularity': '1d',
   'lookback': 100,
   'columns': ['Open', 'High', 'Low', 'Close', 'Volume']}},
 'tickers': ['AAPL', 'MSFT', 'NVDA']}